### Notebook boilerplate

In [15]:
from importlib import reload
from pathlib import Path
import pandas as pd
notebooks_dir = Path('.').resolve()
root_dir = notebooks_dir.parents[0]
import sys
sys.path.append(str(root_dir))
from settings import ACS_YEAR, ACS_SPAN, RAW_ACS_DATA_DIR
import settings
def fix_path(path):
    """Hacky fix to make paths behave when importing from upstream module"""
    return Path(str(path).replace('notebooks/', ''))
RAW_ACS_DATA_DIR = fix_path(RAW_ACS_DATA_DIR)

In [2]:
lookups_src = root_dir / 'acs_5yr_seq_table_number_lookup.csv'
lookups = pd.read_csv(lookups_src, comment='#')
lookups.columns = [x.lower().replace(' ', '_') for x in lookups]
lookups = lookups.query('((get=="1") | (get==1)) & (subject_area.notnull())')
c = ['table_title', 'subject_area']
lookups = lookups[c]

In [19]:
import bs4
import csv
import io
import os
import requests
import sys
import urllib.request
import zipfile




class ACS:
    def __init__(self, acs_year, acs_span, data_dir, overwrite=False):
        self.acs_year = acs_year
        self.acs_span = acs_span
        self.data_dir = Path(data_dir)
        self.lookup_url = f'https://www2.census.gov/programs-surveys/acs/summary_file/{acs_year}/documentation/user_tools/ACS_{acs_span}yr_Seq_Table_Number_Lookup.txt'
        self.data_url = f'https://www2.census.gov/programs-surveys/acs/summary_file/{acs_year}/data/{acs_span}_year_by_state'
        self.lookup_path = self.data_dir / f'{acs_year}_{acs_span}y_lookup.txt'
        self.overwrite = overwrite
        self.data_zips = []
    
    @staticmethod
    def download(src, dst, verbose=True):
        if verbose:
            print('downloading %s -> %s...' % (src, dst), file=sys.stderr)
        urllib.request.urlretrieve(src, dst)
        return True
    
    def get_acs_metadata(self):
        if not self.lookup_path.exists() and self.overwrite == False:
            self.download(self.lookup_url, self.lookup_path)
        return True
    
    def get_acs_data(self):
        # Go to the "data by state" page and scan the HTML page for links to zip files
        soup = bs4.BeautifulSoup(requests.get(self.data_url).content)
        for link in soup.find_all('a'):
            if link.get('href') and link.get('href').endswith('zip'):
                fn = link.get('href').split('/')[-1]
                dst = self.data_dir / fn
                if not dst.exists() and self.overwrite == False:
                    self.download(self.data_url + '/' + fn, dst)
#                 self.data_zips.append(zipfile.ZipFile(os.path.join(self.data_dir, fn), 'r'))
        return True

    def find_table(self, table_title, subject_area):
        with open(self.lookup_fn, 'r', encoding='iso-8859-1') as csvfile:
            reader = csv.DictReader(csvfile, dialect='unix')
            seq_number, start_pos, cells = None, None, []
            current_table_title = None
            for row in reader:
                if row['Table Title'] and row['Total Cells in Table']:
                    current_table_title = row['Table Title']
                if current_table_title == table_title and row['Start Position']:
                    seq_number = int(row['Sequence Number'])
                    start_pos = int(row['Start Position'])
                if current_table_title == table_title and row['Line Number']:
                    try:
                        int(row['Line Number'])
                        cells.append(row['Table Title'])
                    except:
                        pass

        return seq_number, start_pos, cells

    def get_data_zips(self):
        self.data_zips = [zipfile.ZipFile(x, 'r') for x in self.data_dir.iterdir()]
        return True

    def get_geos(self):
        geos = {}
        for data_zip in self.data_zips:
            for info in data_zip.infolist():
                if info.filename.startswith('g') and info.filename.endswith('.csv'):
                    with data_zip.open(info.filename) as csvfile:
                        print('Parsing geography data for', info.filename, file=sys.stderr)
                        data = csvfile.read()
                        buf = io.StringIO(data.decode('iso-8859-1'))
                        reader = csv.reader(buf, dialect='unix')
                        for row in reader:
                            geos[(row[1], row[4])] = row[-4]
        return geos

    def get_table(self, table_title, subject_area):
        seq_number, start_pos, cells = self.find_table(table_title, subject_area)

        ret = {}
        for data_zip in self.data_zips:
            for info in data_zip.infolist():
                if info.filename.startswith('e') and info.filename.endswith('%04d000.txt' % seq_number):
                    with data_zip.open(info.filename) as csvfile:
                        print('Parsing data for', info.filename, file=sys.stderr)
                        data = csvfile.read()
                        buf = io.StringIO(data.decode('iso-8859-1'))
                        reader = csv.reader(buf, dialect='unix')
                        col_i, col_j = start_pos-1, start_pos+len(cells)-1
                        for row in reader:
                            state = row[2].upper()
                            logical_record_number = row[5]
                            values = [int(value) if (value and value != '.' and int(value) > 0) else None for value in row[col_i:col_j]]
                            ret[(state, logical_record_number)] = {k: v for k, v in zip(cells, values) if v is not None}
        return ret


class OneYearACS(ACS):
    def __init__(self):
        super(OneYearACS, self).__init__(
            'https://www2.census.gov/programs-surveys/acs/summary_file/2016/documentation/user_tools/ACS_1yr_Seq_Table_Number_Lookup.txt',
            'acs_lookup.txt',
            'https://www2.census.gov/programs-surveys/acs/summary_file/2016/data/1_year_by_state',
            '1_year_data')


class FiveYearACS(ACS):
    def __init__(self):
        super(FiveYearACS, self).__init__(
            'https://www2.census.gov/programs-surveys/acs/summary_file/2016/documentation/user_tools/ACS_5yr_Seq_Table_Number_Lookup.txt',
            '5y_lookup.txt',
            'https://www2.census.gov/programs-surveys/acs/summary_file/2016/data/5_year_by_state',
            '5_year_data')

In [7]:
reload(settings)


settings.ROOT_DIR

PosixPath('/home/pgr-me/gh/geoclusterizer/notebooks')

In [12]:
reload(settings)

<module 'settings' from '/home/pgr-me/gh/geoclusterizer/settings.py'>

In [113]:
Path.cwd()

PosixPath('/home/pgr-me/gh/geoclusterizer/notebooks')

In [20]:
RAW_DIR = Path('/home/pgr-me/gh/geoclusterizer/data/raw')
lookup_url = f'https://www2.census.gov/programs-surveys/acs/summary_file/{ACS_YEAR}/documentation/user_tools/ACS_{ACS_SPAN}yr_Seq_Table_Number_Lookup.txt'
lookup_fn =  f'{ACS_SPAN}y_lookup.txt'
data_url = f'https://www2.census.gov/programs-surveys/acs/summary_file/{ACS_YEAR}/data/{ACS_SPAN}_year_by_state'
data_dir = settings

acs = ACS(ACS_YEAR, ACS_SPAN, RAW_ACS_DATA_DIR, overwrite=False)
acs.get_acs_metadata()
acs.get_acs_data()




# mortgage_data = ACS().get_table('MORTGAGE STATUS', 'Housing')
# value_data = FiveYearACS().get_table('MEDIAN VALUE (DOLLARS)', 'Housing')
# tax_data = FiveYearACS().get_table('MORTGAGE STATUS BY MEDIAN REAL ESTATE TAXES PAID (DOLLARS)', 'Housing')

# geos = acs.get_geos()
# raise
# for k, name in geos.items():
#     tax = tax_data.get(k, {}).get('Median real estate taxes paid for units with a mortgage')
#     value = value_data.get(k, {}).get('Median value (dollars)')
#     mortgage = mortgage_data.get(k, {}).get('Housing units with a mortgage, contract to purchase, or similar debt:')
#     if tax and value and mortgage:
#         print('%6.2f%% %6d %s' % (100. * tax / value, mortgage, name))

downloading https://www2.census.gov/programs-surveys/acs/summary_file/2018/data/5_year_by_state/Alabama_All_Geographies_Not_Tracts_Block_Groups.zip -> /home/pgr-me/gh/geoclusterizer/data/raw/2018_5_year_data/Alabama_All_Geographies_Not_Tracts_Block_Groups.zip...
downloading https://www2.census.gov/programs-surveys/acs/summary_file/2018/data/5_year_by_state/Alabama_Tracts_Block_Groups_Only.zip -> /home/pgr-me/gh/geoclusterizer/data/raw/2018_5_year_data/Alabama_Tracts_Block_Groups_Only.zip...
downloading https://www2.census.gov/programs-surveys/acs/summary_file/2018/data/5_year_by_state/Alaska_All_Geographies_Not_Tracts_Block_Groups.zip -> /home/pgr-me/gh/geoclusterizer/data/raw/2018_5_year_data/Alaska_All_Geographies_Not_Tracts_Block_Groups.zip...
downloading https://www2.census.gov/programs-surveys/acs/summary_file/2018/data/5_year_by_state/Alaska_Tracts_Block_Groups_Only.zip -> /home/pgr-me/gh/geoclusterizer/data/raw/2018_5_year_data/Alaska_Tracts_Block_Groups_Only.zip...
downloading 

True

In [71]:
acs.get_data()

In [57]:
for i, k in lookups.head(2).iteritems():
    print(i, k)

subject_area table_title
UNWEIGHTED SAMPLE COUNT OF THE POPULATION    Unweighted Count
UNWEIGHTED SAMPLE HOUSING UNITS              Unweighted Count
Name: subject_area, dtype: object
